In [3]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 130.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00


In [4]:
file = open("/content/Trump.txt", "r")

In [5]:
FileContent= file.read().strip()

In [6]:
FileContent


'Thank you. Thank you. Thank you to Vice President Pence. He\'s a good guy. We\'ve done a great job together. And Merry Christmas, Michigan. Thank you, Michigan. What a victory we had in Michigan. What a victory was that. One of the greats. Was that the greatest evening? But I\'m thrilled to be here with thousands of hardworking patriots as we celebrate the miracle of Christmas, the greatness of America and the glory of God. Thank you very much. And did you notice that everybody is saying Merry Christmas again? Did you notice? Saying Merry Christmas. I remember when I first started this beautiful trip, this beautiful journey, I just said to the First Lady, "You are so lucky. I took you on this fantastic journey. It\'s so much fun. They want to impeach you. They want to do worse than that." By the way, by the way, by the way, it doesn\'t really feel like we\'re being impeached. The country is doing better than ever before. We did nothing wrong. We did nothing wrong. And we have tremendo

In [7]:
len(FileContent)

95916

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint="facebook/bart-large-cnn"

tokenizer= AutoTokenizer.from_pretrained(checkpoint)
model= AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [9]:
 tokenizer.model_max_length

1024

In [10]:
tokenizer.max_len_single_sentence

1022

In [11]:
tokenizer.num_special_tokens_to_add()

2

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
sentences= nltk.tokenize.sent_tokenize(FileContent)

In [14]:
len(sentences)

1893

##max tokens in a sentence


In [15]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

64

In [16]:
length= 0
chunk =""
chunks=[]
count=-1

for sentence in sentences:
  count+=1
  combined_length = len(tokenizer.tokenize(sentence)) + length

  if combined_length<= tokenizer.max_len_single_sentence:
    chunk+= sentence + " "
    length= combined_length

    if count == len(sentence) -1:
      chunks.append(chunk.strip())
    
  else:
    chunks.append(chunk.strip())

    length=0
    chunk= ""

    chunk += sentence + " "
    length= len(tokenizer.tokenize(sentence))

len(chunks)

26

In [17]:
[len(tokenizer.tokenize(c)) for c in chunks]


[117,
 363,
 760,
 1017,
 1019,
 987,
 1008,
 1020,
 1020,
 981,
 977,
 985,
 1003,
 1015,
 1014,
 1010,
 994,
 1018,
 1009,
 1009,
 1016,
 1006,
 1016,
 1004,
 1017,
 1011]

In [18]:
[len(tokenizer(c).input_ids) for c in chunks]

[119,
 365,
 762,
 1019,
 1021,
 989,
 1010,
 1022,
 1022,
 983,
 979,
 987,
 1005,
 1017,
 1016,
 1012,
 996,
 1020,
 1011,
 1011,
 1018,
 1008,
 1018,
 1006,
 1019,
 1013]

In [19]:
sum([len(tokenizer(c).input_ids) for c in chunks])

24448

In [20]:
len(tokenizer.tokenize(FileContent))

Token indices sequence length is longer than the specified maximum sequence length for this model (23182 > 1024). Running this sequence through the model will result in indexing errors


23182

In [22]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
for input in inputs:
  output=model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Thank you. Thank you to Vice President Pence. We've done a great job together. And Merry Christmas, Michigan. What a victory we had in Michigan. One of the greats. Was that the greatest evening? But I'm thrilled to be here with thousands of hardworking patriots as we celebrate the miracle of Christmas.
The president was in Michigan to celebrate Christmas with thousands of hardworking patriots. Trump: "The country is doing better than ever before. We did nothing wrong. And we have tremendous support in the Republican Party like we've never had before" "It doesn't really feel like we're being impeached," he said.
The president is in Michigan to celebrate Christmas with thousands of hardworking patriots. Trump: "The country is doing better than ever before. The country is thriving and it's thriving truly like it has never, it's never happened before to the extent what's happening now"
The president is in Michigan, where he was named the Man Of The Year 10 years ago. He says the country is